In [1]:
import Arena
from MCTS import MCTS

from tictactoe.TicTacToeGame import TicTacToeGame as Game
from tictactoe.keras.NNet import NNetWrapper as NNet
from utils import *
import numpy as np

/Users/gianluigimucciolo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
MODEL_FOLDER = './pretrained_models/tictactoe/'
MODEL_NAME = 'padawan0.network'

In [3]:
game = Game(3)
nn = NNet(game)
nn.load_checkpoint(MODEL_FOLDER,MODEL_NAME)
args1 = dotdict({'numMCTSSims': 50, 'cpuct':1.0})
mcts1 = MCTS(game, nn, args1)
network = lambda x: np.argmax(mcts1.getActionProb(x, temp=0))

In [4]:
def display(board, valids, currPlayer, init_board, base_api, message=None):
    row,col = board.shape
    moves = []
    for i in range(len(valids)):
        if valids[i]:
            moves.append((int(i/row), int(i%col)))
    html = '<table border="1">'
    for r in range(0,row):
        html += '<tr>'
        for c in range(0,col):
            piece = board[r][c]
            if piece == -1: html += '<th>{}</th>'.format("X")
            elif piece == 1: html += '<th>{}</th>'.format("O")
            else:
                hfer = '----'
                if (r,c) in moves:
                    next_board = np.copy(board)
                    next_board[r][c] = -1 #Player Simbol
                    query = ','.join([str(e) for e in next_board.reshape(-1)]) + ',' + currPlayer
                    hfer = '<a href="{}?board={}">{}</a>'.format(base_api, query, hfer)
                html += '<th>{}</th>'.format(hfer)               
        html += '</tr>' 
    html += '</table>'
    html += '<br>{}<br><a href="{}?board={}">Start Over!</a>'.format(message, base_api, '{},{}'.format(init_board, currPlayer)) if message else ""
    return html

In [17]:
def playGame(board, init_board = '0,0,0,0,0,0,0,0,0', base_api = 'localhost'):
    if len(board) == 0:
        board = '{},N'.format(init_board)
        
    board = board.split(',') 
    currPlayer = board[-1]
    board = np.array(board[:-1]).reshape(3,3).astype(int)
  
    status = game.getGameEnded(board, 1)
    if status != 0:
        message = "You Lose" if status == 1 else ("You Win" if status == -1 else "Draw.")
        currPlayer = "H" if currPlayer == "N" else "N"
        return display(board, [], currPlayer, init_board, base_api, message)

    action = network(game.getCanonicalForm(board, 1))
    valids = game.getValidMoves(game.getCanonicalForm(board, 1),1)
    
    if np.sum(board) == 0 and currPlayer == "H":
        return display(board, valids, currPlayer, init_board, base_api)
    
    board, _ = game.getNextState(board, 1, action)                
    valids = game.getValidMoves(game.getCanonicalForm(board, 1),1)
    return display(board, valids, currPlayer, init_board, base_api)

In [6]:
board = '0,0,0,0,0,0,0,0,0,N'
playGame(board)

'<table border="1"><tr><th><a href="localhost?board=-1,0,0,0,1,0,0,0,0,N">----</a></th><th><a href="localhost?board=0,-1,0,0,1,0,0,0,0,N">----</a></th><th><a href="localhost?board=0,0,-1,0,1,0,0,0,0,N">----</a></th></tr><tr><th><a href="localhost?board=0,0,0,-1,1,0,0,0,0,N">----</a></th><th>O</th><th><a href="localhost?board=0,0,0,0,1,-1,0,0,0,N">----</a></th></tr><tr><th><a href="localhost?board=0,0,0,0,1,0,-1,0,0,N">----</a></th><th><a href="localhost?board=0,0,0,0,1,0,0,-1,0,N">----</a></th><th><a href="localhost?board=0,0,0,0,1,0,0,0,-1,N">----</a></th></tr></table>'

In [7]:
board = '-1,0,0,0,1,0,0,0,0,N'
playGame(board)

'<table border="1"><tr><th>X</th><th><a href="localhost?board=-1,-1,1,0,1,0,0,0,0,N">----</a></th><th>O</th></tr><tr><th><a href="localhost?board=-1,0,1,-1,1,0,0,0,0,N">----</a></th><th>O</th><th><a href="localhost?board=-1,0,1,0,1,-1,0,0,0,N">----</a></th></tr><tr><th><a href="localhost?board=-1,0,1,0,1,0,-1,0,0,N">----</a></th><th><a href="localhost?board=-1,0,1,0,1,0,0,-1,0,N">----</a></th><th><a href="localhost?board=-1,0,1,0,1,0,0,0,-1,N">----</a></th></tr></table>'

In [8]:
board = '-1,0,1,0,1,0,0,-1,0,N'
playGame(board)

'<table border="1"><tr><th>X</th><th><a href="localhost?board=-1,-1,1,0,1,0,1,-1,0,N">----</a></th><th>O</th></tr><tr><th><a href="localhost?board=-1,0,1,-1,1,0,1,-1,0,N">----</a></th><th>O</th><th><a href="localhost?board=-1,0,1,0,1,-1,1,-1,0,N">----</a></th></tr><tr><th>O</th><th>X</th><th><a href="localhost?board=-1,0,1,0,1,0,1,-1,-1,N">----</a></th></tr></table>'

In [9]:
board = '-1,0,1,-1,1,0,1,-1,0,N'
playGame(board)

'<table border="1"><tr><th>X</th><th>----</th><th>O</th></tr><tr><th>X</th><th>O</th><th>----</th></tr><tr><th>O</th><th>X</th><th>----</th></tr></table><br>You Lose<br><a href="localhost?board=0,0,0,0,0,0,0,0,0,H">Start Over!</a>'

In [10]:
board = '0,0,0,0,0,0,0,0,0,H'
playGame(board)

'<table border="1"><tr><th><a href="localhost?board=-1,0,0,0,0,0,0,0,0,H">----</a></th><th><a href="localhost?board=0,-1,0,0,0,0,0,0,0,H">----</a></th><th><a href="localhost?board=0,0,-1,0,0,0,0,0,0,H">----</a></th></tr><tr><th><a href="localhost?board=0,0,0,-1,0,0,0,0,0,H">----</a></th><th><a href="localhost?board=0,0,0,0,-1,0,0,0,0,H">----</a></th><th><a href="localhost?board=0,0,0,0,0,-1,0,0,0,H">----</a></th></tr><tr><th><a href="localhost?board=0,0,0,0,0,0,-1,0,0,H">----</a></th><th><a href="localhost?board=0,0,0,0,0,0,0,-1,0,H">----</a></th><th><a href="localhost?board=0,0,0,0,0,0,0,0,-1,H">----</a></th></tr></table>'

In [11]:
board = '-1,0,0,0,0,0,0,0,0,H'
playGame(board)

'<table border="1"><tr><th>X</th><th><a href="localhost?board=-1,-1,0,0,1,0,0,0,0,H">----</a></th><th><a href="localhost?board=-1,0,-1,0,1,0,0,0,0,H">----</a></th></tr><tr><th><a href="localhost?board=-1,0,0,-1,1,0,0,0,0,H">----</a></th><th>O</th><th><a href="localhost?board=-1,0,0,0,1,-1,0,0,0,H">----</a></th></tr><tr><th><a href="localhost?board=-1,0,0,0,1,0,-1,0,0,H">----</a></th><th><a href="localhost?board=-1,0,0,0,1,0,0,-1,0,H">----</a></th><th><a href="localhost?board=-1,0,0,0,1,0,0,0,-1,H">----</a></th></tr></table>'

In [12]:
board = '-1,0,0,0,1,-1,0,0,0,H'
playGame(board)

'<table border="1"><tr><th>X</th><th><a href="localhost?board=-1,-1,1,0,1,-1,0,0,0,H">----</a></th><th>O</th></tr><tr><th><a href="localhost?board=-1,0,1,-1,1,-1,0,0,0,H">----</a></th><th>O</th><th>X</th></tr><tr><th><a href="localhost?board=-1,0,1,0,1,-1,-1,0,0,H">----</a></th><th><a href="localhost?board=-1,0,1,0,1,-1,0,-1,0,H">----</a></th><th><a href="localhost?board=-1,0,1,0,1,-1,0,0,-1,H">----</a></th></tr></table>'

In [13]:
board = '-1,0,1,0,1,-1,-1,0,0,H'
playGame(board)

'<table border="1"><tr><th>X</th><th><a href="localhost?board=-1,-1,1,1,1,-1,-1,0,0,H">----</a></th><th>O</th></tr><tr><th>O</th><th>O</th><th>X</th></tr><tr><th>X</th><th><a href="localhost?board=-1,0,1,1,1,-1,-1,-1,0,H">----</a></th><th><a href="localhost?board=-1,0,1,1,1,-1,-1,0,-1,H">----</a></th></tr></table>'

In [14]:
board = '-1,0,1,1,1,-1,-1,-1,0,H'
playGame(board)

'<table border="1"><tr><th>X</th><th><a href="localhost?board=-1,-1,1,1,1,-1,-1,-1,1,H">----</a></th><th>O</th></tr><tr><th>O</th><th>O</th><th>X</th></tr><tr><th>X</th><th>X</th><th>O</th></tr></table>'

In [15]:
board = '-1,-1,1,1,1,-1,-1,-1,1,H'
playGame(board)

'<table border="1"><tr><th>X</th><th>X</th><th>O</th></tr><tr><th>O</th><th>O</th><th>X</th></tr><tr><th>X</th><th>X</th><th>O</th></tr></table><br>Draw.<br><a href="localhost?board=0,0,0,0,0,0,0,0,0,N">Start Over!</a>'

In [16]:
board = '0,0,0,0,0,0,0,0,0,N'
playGame(board)

'<table border="1"><tr><th><a href="localhost?board=-1,0,0,0,1,0,0,0,0,N">----</a></th><th><a href="localhost?board=0,-1,0,0,1,0,0,0,0,N">----</a></th><th><a href="localhost?board=0,0,-1,0,1,0,0,0,0,N">----</a></th></tr><tr><th><a href="localhost?board=0,0,0,-1,1,0,0,0,0,N">----</a></th><th>O</th><th><a href="localhost?board=0,0,0,0,1,-1,0,0,0,N">----</a></th></tr><tr><th><a href="localhost?board=0,0,0,0,1,0,-1,0,0,N">----</a></th><th><a href="localhost?board=0,0,0,0,1,0,0,-1,0,N">----</a></th><th><a href="localhost?board=0,0,0,0,1,0,0,0,-1,N">----</a></th></tr></table>'